In [2]:
from scipy import stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
%matplotlib inline

# Part 1

In [5]:
crime_data=pd.read_csv("Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv")
crime_data

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,146196161,NON-CRIMINAL,LOST PROPERTY,Tuesday,09/23/2014,01:00,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403405,37.775421,POINT (-122.403404791479 37.775420706711),14619616171000
1,150045675,ASSAULT,BATTERY,Thursday,01/15/2015,17:00,TARAVAL,NONE,1800 Block of VICENTE ST,-122.485604,37.738821,POINT (-122.48560378101 37.7388214326705),15004567504134
2,140632022,SUSPICIOUS OCC,INVESTIGATIVE DETENTION,Wednesday,07/30/2014,09:32,BAYVIEW,NONE,100 Block of GILLETTE AV,-122.396535,37.710660,POINT (-122.396535107224 37.7106603302503),14063202264085
3,150383259,ASSAULT,BATTERY,Saturday,05/02/2015,23:10,BAYVIEW,"ARREST, BOOKED",2400 Block of PHELPS ST,-122.400131,37.730093,POINT (-122.400130573297 37.7300925390327),15038325904134
4,40753980,OTHER OFFENSES,RECKLESS DRIVING,Friday,07/02/2004,13:43,BAYVIEW,NONE,I-280 / CESAR CHAVEZ ST,-120.500000,90.000000,POINT (-120.5 90),4075398065020
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2215019,160648742,NON-CRIMINAL,AIDED CASE,Thursday,08/11/2016,18:36,SOUTHERN,NONE,0 Block of WASHBURN ST,-122.414406,37.775236,POINT (-122.41440629479061 37.77523622314656),16064874251040
2215020,160659761,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Monday,08/15/2016,19:06,BAYVIEW,"ARREST, BOOKED",16TH ST / 3RD ST,-122.389103,37.766904,POINT (-122.38910303281254 37.76690372519212),16065976165016
2215021,170285083,VEHICLE THEFT,STOLEN TRUCK,Thursday,04/06/2017,23:30,PARK,NONE,800 Block of HAIGHT ST,-122.436214,37.771317,POINT (-122.43621442698411 37.77131718410027),17028508307025
2215022,160684605,LOITERING,LODGING IN PARK,Wednesday,08/24/2016,07:33,NORTHERN,NONE,1600 Block of BAY ST,-122.434155,37.802896,POINT (-122.43415507683311 37.80289596016211),16068460519062


In [6]:
focuscrimes = set(['WEAPON LAWS', 'PROSTITUTION', 'DRIVING UNDER THE INFLUENCE', 'ROBBERY', 'BURGLARY', 'ASSAULT', 'DRUNKENNESS', 'DRUG/NARCOTIC', 'TRESPASS', 'LARCENY/THEFT', 'VANDALISM', 'VEHICLE THEFT', 'STOLEN PROPERTY', 'DISORDERLY CONDUCT'])

#  Part 2

# Part 3

# Part 4

# Part 5

# Part 6